In [1]:
!pip install transformers
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 226188, done.
remote: Counting objects: 100% (41335/41335), done.
remote: Compressing objects: 100% (2992/2992), done.
remote: Total 226188 (delta 40439), reused 38347 (delta 38343), pack-reused 184853 (from 1)
Receiving objects: 100% (226188/226188), 228.37 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (165720/165720), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9614317 sha256=c301bdf3de1f5f0ac72ba81253105868e676a567704efa7b253c86c0d7076393
  Stored in directory: /tmp/pip-ephem-wheel-cache-d5vl2wwn/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
  

In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install fugashi ipadic

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

In [3]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

text = '父の父は、祖父。'
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

['父', 'の', '父', 'は', '、', '祖父', '。']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 2
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)


['父', 'の', '[MASK]', 'は', '、', '祖父', '。']
[800, 5, 4, 9, 6, 5235, 8]
tensor([[ 800,    5,    4,    9,    6, 5235,    8]])


In [5]:
# Load pre-trained model
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model.eval()

# Predict
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0][0, masked_index].topk(5) # 予測結果の上位5件を抽出

# Show results
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 の
1 、
2 と
3 父
4 母


In [6]:
from transformers import BertJapaneseTokenizer, BertModel
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model_bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model_bert.eval()

import numpy as np

def calc_embedding(text):
  bert_tokens = tokenizer.tokenize(text)
  ids = tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
  tokens_tensor = torch.tensor(ids).reshape(1, -1)

  with torch.no_grad():
    output = model_bert(tokens_tensor)

  return output[1].numpy()

text = '私はからあげです'
calc_embedding(text)


array([[-4.80718687e-02,  4.65254039e-01, -1.83068931e-01,
        -3.77458006e-01,  9.60653573e-02,  9.98048246e-01,
        -7.86745250e-02, -1.10579140e-01, -2.16161653e-01,
         2.70328224e-01,  6.12610102e-01, -5.22121079e-02,
        -9.10361949e-03, -2.11077571e-01,  1.13648802e-01,
        -1.01962671e-01,  1.15808062e-02,  9.70591605e-02,
         2.84477472e-01,  1.56308711e-01,  9.21595395e-02,
         3.41767728e-01, -3.17928970e-01,  9.66841817e-01,
         9.99929190e-01,  1.15791321e-01,  4.14485484e-01,
        -5.67455173e-01,  9.99083221e-01,  8.55081499e-01,
        -7.46547997e-01,  4.08208668e-01, -8.66946876e-02,
         3.22804928e-01,  3.65310371e-01,  3.17443758e-01,
         1.38259009e-01, -2.39119202e-01, -3.83762866e-01,
         1.85939938e-01,  1.97875742e-02, -9.98861074e-01,
        -3.62229586e-01, -6.72741011e-02,  7.23658085e-01,
         1.12314366e-01,  4.84685481e-01, -2.40226954e-01,
         2.71576673e-01, -1.90292597e-01, -3.45356092e-0